# 3. Binding energy computation

At this point we have already sampled one or more amorphized water clusters that model a ice-grain mantel 
surface and are ready to compute the binding energy on those binding sites. For that purpose  
we use a second CLI tool called `launch_energy.py`  that is contained in the scripts folder. The options for `launch_energy` are the following:

```
Usage: python launch_energy_new.py [options]

A command line interface to compute the binding energy of a set of optimized binding sites
on a set of water clusters (stored in a 
QCFractal DataSet). This CLI is part of the Binding Energy Evaluation Platform (BEEP).


Options:
  -h, --help            show this help message and exit
  --cluster_collection=CLUSTER_COLLECTION
                        The name of the collection with the water clusters
                        (default: Water_22)
  --small_mol_collection=SMALL_MOLECULE_COLLECTION
                        The name of the collection with small molecules or
                        radicals (default: Small_molecules)
  --molecule=MOLECULE   The name of the molecule for the binding energy
                        computation
  --level_of_theory=LEVEL_OF_THEORY
                        The level of theory for the binding energy computation
                        in the format: method_basis (default:
                        wpbe-d3bj_def2-tzvp)
  --opt_level_of_theory=OPT_LOT
                        The level of theory of for the optimization of the
                        binding sites in the format: method_basis (default:
                        hf3c_minix)
  --keyword_id=KEYWORD_ID
                        ID of the QCfractal for the single point computations
                        keywords (default: None)
  --hessian_compute=HESSIAN
                        Computes the hessian for the molecules comprising the
                        binding sites of model cluster X. If 0 is specified,
                        no Hessian is computed (defualt = 1)
  -p PROGRAM, --program=PROGRAM
                        The program to use for this calculation (default:
                        psi4)
```

Here it is important to note that you have to specify both the level of theory at  which you would like to do the binding energy computation and also the level of theory at which you optimized the different binding sites. Also note that the isolated  monomers have to be optimized at the same level of theory as the supermolecule. For example if  you want to compute the binding energy for binding sites optimized at a HF-3C/MINIX level of theory, the geometry of the  isolated water clusters and target molecules need to exist at that level of theory, in our  example we can compute them  for the water_12 set of clusters and the small molecule collection:






In [11]:
import qcfractal.interface as ptl

client = ptl.FractalClient(address="localhost:7777", verify=False)

In [5]:
ds_w12 = client.get_collection("OptimizationDataset", "water_12")
ds_sm = client.get_collection("OptimizationDataset", "small_molecules")

In [6]:
spec = add_spec = {'name': 'hf3c_minix',
        'description': 'Geometric + Psi4/hf3c/minix.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'hf3c',
        'basis': 'minix',
        'keywords': None,
        'program': 'psi4'}}
ds_w12.add_specification(**add_spec,overwrite=True)
ds_w12.save()
ds_sm.add_specification(**add_spec,overwrite=True)
ds_sm.save()

'72'

In [5]:
ds_sm.compute('hf3c_minix', tag='basic')

1

In [7]:
ds_w12.compute('hf3c_minix', tag='basic')

5

The binding energy computation program will search for  all unique binding sites on the different water clusters
and compute the binding energy for those binding sites. These will be stored in a so called
`ReactionDataset` where the BSSE corrected stoichiometry is automatically stored. There is also an option
to compute the Hessian matrix  for all binding sites on one water cluster of the users choice. This is 
necessary in order to construct a model for the ZPVE correction 
that can be applied to all the binding energies within a model (see 04_Hessian_and_ZPVE). 

**IMPORTANT**: Before running `launch_energy.py` you need to spin up qcfractal-managers, one with a `tag=comp_be` and a second with a `tag=comp_hessian` (if requested). This module contains two examples of qcfractal-managers running using dask and slurm for this specific purpose. For more examples and options check out  [QCFractal manager documentation ](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html)

Form the ReactionDataset you can also retrieve all type of binding sites energy data like
interaction energies and deformation energies!  In order to check the computed binding energies 
you can call each individual water cluster from the server. First we call the client

Then we call the `ReactionDataset`. The naming convention of the collections that store the binding energies
is the same as the in the `OptimizationDataset` but with a `be` prefix and the QC method with which the binding site was optimized as a suffix. So in our example we need to call: `be_ch3oh_W12_1_hf3c`:

In [15]:
ds_be = client.get_collection("ReactionDataset", 'be_ch3oh_W12_3_hf3c')

Now you can print the values of the binding energies with the `get_value()` methods by specifying the required 
stoichiometry and the QC method for which the binding energy values where computed. The default is a BSSE corrected binding energy:

In [16]:
ds_be.get_values(stoich='default',method="B3LYP-D3BJ")

,B3LYP-D3BJ/def2-svp,B3LYP-D3BJ/def2-tzvp
ch3oh_w12_3_0001,-9.62136,-8.907313
ch3oh_w12_3_0002,-11.302769,-10.245449
ch3oh_w12_3_0003,-7.414099,-9.917144
ch3oh_w12_3_0006,-9.349479,-8.737617
ch3oh_w12_3_0008,-12.255689,-10.927993
ch3oh_w12_3_0012,-11.550165,-10.799472
ch3oh_w12_3_0016,-9.863477,-11.379328
ch3oh_w12_3_0021,-8.580385,-8.00788
ch3oh_w12_3_0023,-8.679396,-8.113023
ch3oh_w12_3_0028,-7.958984,-6.923595


However it is also possible to obtain binding energies that are not counterpoise corrected (`stoich=be_nocp`) 
interaction energies (`stoich=int`) and deformation energies (`stoich=de`)

Finally, we recommend to run `launch_energy` in the background as it automatically detects if the binding sites of a  new cluster have been fully optimized and will stop once all the BE computations of all clusters in a given set have been sent. To monitor the progress, a log file is updated in every cycle. 